<a href="https://colab.research.google.com/github/blackrosedragon2/ReccomenderSystem/blob/master/ContentBasedFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload() #uploads file from local computer

Saving AnimeList.csv to AnimeList.csv


In [0]:
#step 1 make genre list


In [0]:
import io
import pandas as pd
import numpy as np
import re
%xmode Plain
df2 = pd.read_csv(io.BytesIO(uploaded['AnimeList.csv']))# converts the uploaded file into pandas format
df2["genre"]=df2["genre"].fillna("o")
df2=df2.fillna(0)
gtable=df2["genre"]
for gtype,row in zip(gtable,df2.iterrows()):
  if type(gtype)!=type("nan"):
    df2=df2.drop(df2.index[row]) #deleting all panda rows which have non string genres
df2=df2[df2.rating>=8]
df2=df2[df2.members>500]  # removing all shows with members <500
df2=df2[df2.genre!="o"]
gtable=df2["genre"]
genre_list=[]
dropped_rows=[]
row=-1
for gname in gtable: # gname is variable that takes each genre row as a string 
  row=row+1
  try:
    genre_list.append(gname.split(","))  # converting the genre row ex:[Drama, Romance, School, Supernatural] to ['Drama','Romance','School','Supernatural']	with the split function
  except:
    df2.drop(df2.index[row]) #incase an exception occurs 
    dropped_rows.append(row)
 #NOTE : genre_list should have actually been named genre_feature_list because only features are contained within it
genre_list = [item for sublist in genre_list for item in sublist] # converting nested list into a single list (ie flattened the list)
genre_list=[i.strip(' ') for i in genre_list] #removes all blanks from each string
genre_list= list(dict.fromkeys(genre_list)) #converting the list into a dictionary that arranges it in ascending order and then converting that dictionary back into a list
genre_list.sort() #Sorting alphabetically

d8=[] #for getting dates
for date in df2["aired_string"]:
  lis=(re.findall(r'[0-9]+', date))
  flag=0
  for l in lis:
    if int(l)<2100 and int(l)>1900:
      d8.append(int(l))
      flag=1
      break
  if flag==0:
    d8.append(0)
df2["aired_string"]=d8
df2

Exception reporting mode: Plain


,anime_ids,name,type,source,episodes,aired_string,duration,age rating,rating,rank,popularity,members,favorites,related,studio,genre
7,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,2009,24 min. per ep.,R - 17+ (violence & profanity),9.25,1.0,4,1199091,106895,"{'Adaptation': [{'mal_id': 25, 'type': 'manga'...",Bones,"Action, Military, Adventure, Comedy, Drama, Ma..."
9,32281,Kimi no Na wa.,Movie,Original,1,2016,1 hr. 46 min.,PG-13 - Teens 13 or older,9.19,2.0,33,730076,34912,"{'Adaptation': [{'mal_id': 99314, 'type': 'man...",CoMix Wave Films,"Supernatural, Drama, Romance, School"
10,28977,Gintama°,TV,Manga,51,2015,24 min. per ep.,R - 17+ (violence & profanity),9.15,3.0,383,198744,5668,"{'Adaptation': [{'mal_id': 44, 'type': 'manga'...",Bandai Namco Pictures,"Action, Comedy, Historical, Parody, Samurai, S..."
11,30484,Steins;Gate 0,TV,Visual novel,23,2018,23 min. per ep.,PG-13 - Teens 13 or older,9.15,4.0,323,223989,1882,"{'Adaptation': [{'mal_id': 107548, 'type': 'ma...",White Fox,"Sci-Fi, Thriller"
12,9253,Steins;Gate,TV,Visual novel,24,2011,24 min. per ep.,PG-13 - Teens 13 or older,9.14,5.0,8,1010330,92423,"{'Adaptation': [{'mal_id': 17517, 'type': 'man...",White Fox,"Thriller, Sci-Fi"
13,11061,Hunter x Hunter (2011),TV,Manga,148,2011,23 min. per ep.,PG-13 - Teens 13 or older,9.11,8.0,35,720920,64375,"{'Adaptation': [{'mal_id': 26, 'type': 'manga'...",Madhouse,"Action, Adventure, Super Power, Shounen"
14,9969,Gintama&#039;,TV,Manga,51,2011,24 min. per ep.,PG-13 - Teens 13 or older,9.11,7.0,342,215527,4549,"{'Adaptation': [{'mal_id': 44, 'type': 'manga'...",Sunrise,"Action, Sci-Fi, Comedy, Historical, Parody, Sa..."
15,820,Ginga Eiyuu Densetsu,OVA,Novel,110,1988,26 min. per ep.,R - 17+ (violence & profanity),9.11,6.0,671,124993,8567,"{'Adaptation': [{'mal_id': 6978, 'type': 'mang...","Artland, Magic Bus","Military, Sci-Fi, Space, Drama"
16,35180,3-gatsu no Lion 2nd Season,TV,Manga,22,2017,25 min. per ep.,PG-13 - Teens 13 or older,9.10,9.0,990,85744,2257,"{'Adaptation': [{'mal_id': 1224, 'type': 'mang...",Shaft,"Drama, Game, Seinen, Slice of Life"
17,15417,Gintama&#039;: Enchousen,TV,Manga,13,2012,24 min. per ep.,PG-13 - Teens 13 or older,9.07,10.0,685,123572,1509,"{'Adaptation': [{'mal_id': 44, 'type': 'manga'...",Sunrise,"Action, Comedy, Historical, Parody, Samurai, S..."


In [0]:
#step 2 create dataframe with anime ids in sorted order along with their genres

In [0]:
itable=df2["anime_ids"]
ttable=df2["type"]
etable=df2["episodes"]
rtable=df2["rating"]
datetable=df2["aired_string"]
ilist=[]
tlist=[]
elist=[]
glist=[]
rlist=[]
datelist=[]
row=-1
def date_class(date):
  '''
  CRITERIA FOR TYPE OF SHOW:
  6: 2009-current
  5: 2004-2009
  4: 1998-2004
  3: 1990-1998
  2: 1980-1990
  1: <1980
  '''
  datenum=0
  if date>=2009:
    datenum=6
  elif date<2009 and date>=2004:
    datenum=5
  elif date<2004 and date>=1998:
    datenum=4
  elif date<1998 and date>=1990:
    datenum=3
  elif date<1990 and date>=1980:
    datenum=2
  else:
    datenum=1
  return datenum
  
def type_class(tname):
  '''
  CRITERIA FOR TYPE OF SHOW:
  1: ONA
  2: Music
  3: Special
  4: OVA
  5. Manga
  6: Movie
  7. Original
  8: TV
  '''
  typenum=0
  if tname=="ONA":
    typenum=1
  elif tname=="Music":
    typenum=2
  elif tname=="Special":
    typenum=3
  elif tname=="OVA":
    typenum=4
  elif tname=="Manga":
    typenum=5
  elif tname=="Movie":
    typenum=6
  elif tname=="Original":
    typenum=7
  else:
    typenum=8
  return typenum
  
def get_series_length(ename):
  '''
  CRITERA FOR EPISODE LENGTH:
  <=13 eps Very Short series - 1
  <=25 eps Short series -2
  <=64 eps Medium series -3
  <=200 eps Long series -4
  <=500 eps Very Long series -5
  >500 eps Extremly Long series -6
  '''
  slen=int(ename)
  if slen<=13:
    slen=1
  elif slen<=25 and slen>13:
    slen=2
  elif slen<=64 and slen>25:
    slen=3
  elif slen<=200 and slen>64:
    slen=4
  elif slen<=500 and slen>200:
    slen=5
  else:
    slen=6
  return slen
  
for (iname,tname,ename,gname,rnum,date) in zip(itable,ttable,etable,gtable,rtable,datetable):
  row=row+1
  if ~(any(row==i for i in dropped_rows)): #goes through entire list of dropped rows and compares them with the current rows, if cond applies if current row is not dropped
    rlist.append(rnum)
    ilist.append(iname) 
    glist.append(gname)#contains genres row wise 
    slen=get_series_length(ename)
    elist.append(slen)
    typenum=type_class(tname)
    tlist.append(typenum)
    datelist.append(date_class(date))

feature_list=["Anime_Id","Rating","Type","Episodes","Date"]+genre_list


In [0]:
data=[]
content_filter=pd.DataFrame(data,columns=feature_list)
content_filter

,Anime_Id,Rating,Type,Episodes,Date,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire


In [0]:
#Step 3: Fill data in each row according to the anime ID (1 for present , 0 for not present)
#data given : glist , ilist , tlist , elist

In [0]:
#WARNING RUNNING THIS WILL TAKE A LONG TIME (at least 2-3 mins)
row=-1
for (iname,tname,ename,rnum,date) in zip(ilist,tlist,elist,rlist,datelist):
  row=row+1
  app_frame_temp=pd.DataFrame({"Anime_Id":iname,"Rating":rnum,"Type":tname,"Episodes":ename,"Date":date},index=[row])
  content_filter=pd.concat([content_filter,app_frame_temp],sort=False) 



In [0]:
#WARNING RUNNING THIS WILL TAKE A SOME TIME (at least 1-2 mins)
row=-1
maxlen=0
penalty=[]
for i in range(13): #generate tanh penalties for the first 13 genre(len)
  penalty.append(np.tanh(((i+1)/10 )- 0.1))
print(penalty)
for gname in glist: #glist contains row wise genres 
  row=row+1
  try:
    fivegen=gname.split(',')# Converting gname to a list
    fivegen=[i.strip(' ') for i in fivegen]  # removing all blank spaces form fivegen
    if maxlen<len(fivegen):
      maxlen=len(fivegen)
    for i in range(len(fivegen)): #Take maximum upto 3 genres/ anime , so no anime gets a clear advantage while getting reccomended 
      try:
        rowIndex = content_filter.index[row]  
        content_filter.loc[rowIndex,fivegen[i]] = 1-penalty[len(fivegen)-1]# replaing a particular row,column with 1 in content_filter dataframe
        # 1-penalty , penalty for the number of genres that will affect the reccomender system
      except: # incase fivegen[i] goes out of bounds 
        break

  except:
    df2.drop(df2.index[row]) # incase NAN values pop up
content_filter=content_filter.fillna(0) #filling all nan values
content_filter

[0.0, 0.09966799462495582, 0.19737532022490398, 0.2913126124515909, 0.3799489622552249, 0.46211715726000974, 0.5370495669980353, 0.6043677771171636, 0.6640367702678491, 0.7162978701990245, 0.7615941559557649, 0.8004990217606296, 0.8336546070121552]


,Anime_Id,Rating,Type,Episodes,Date,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire
0,5114,9.25,8,3,6,0.395632,0.395632,0.0,0.395632,0.000000,0.0,0.395632,0.00000,0.395632,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.395632,0.000000,0.000000,0.395632,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.395632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,32281,9.19,6,1,6,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.708687,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.708687,0.000000,0.708687,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.708687,0.000000,0.000000
2,28977,9.15,8,3,6,0.462950,0.000000,0.0,0.462950,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.462950,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.46295,0.000000,0.000000,0.000000,0.462950,0.000000,0.462950,0.000000,0.000000,0.0,0.462950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,30484,9.15,8,2,6,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900332,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900332,0.000000
4,9253,9.14,8,2,6,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900332,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900332,0.000000
5,11061,9.11,8,4,6,0.708687,0.708687,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.708687,0.000000,0.000000,0.000000,0.000000,0.708687,0.000000,0.000000,0.000000
6,9969,9.11,8,3,6,0.462950,0.000000,0.0,0.462950,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.462950,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.46295,0.000000,0.000000,0.000000,0.462950,0.000000,0.462950,0.000000,0.000000,0.0,0.462950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,820,9.11,4,4,2,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.708687,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.708687,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.708687,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.708687,0.000000,0.000000,0.000000,0.000000,0.000000
8,35180,9.10,8,2,6,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.708687,0.00000,0.000000,0.708687,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.708687,0.000000,0.0,0.000000,0.000000,0.708687,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,15417,9.07,8,1,6,0.462950,0.000000,0.0,0.462950,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.462950,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.46295,0.000000,0.000000,0.000000,0.462950,0.000000,0.462950,0.000000,0.000000,0.0,0.462950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [0]:
!mkdir .kaggle


In [0]:
import json
token ={"username":"blackrosedragon2","key":"f6b2d34630e576848cb1ef4cb1b6501a"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json


!kaggle datasets download -d azathoth42/myanimelist -p /content #download kaggle


100% 1.92G/1.92G [00:27<00:00, 92.2MB/s]
100% 1.92G/1.92G [00:27<00:00, 73.8MB/s]


In [0]:
# i am trying to figure out what my_status field is as well as how i will seperate the users in a dataframe
#sample data frame from user list
import zipfile
'''
my_status in animelists tables contains integer values. This is their semantics:

1: watching
2: completed
3: on hold
4: dropped
6: plan to watch
'''
def get_user_data(username):
  j=10
    
  user=pd.DataFrame()
  zf = zipfile.ZipFile('myanimelist.zip')
  for chunk in pd.read_csv(zf.open('UserAnimeList.csv'), chunksize=100000):
    df=pd.DataFrame(chunk)
    if any(df["username"]==username):
      user_particular=(df[df["username"]==username])
      user = user.append(user_particular[["anime_id","my_watched_episodes","my_score","my_rewatching","my_status"]],sort=False)
      if j==0:
        break
      j=j-1
    if j<10:
      break
  return user
def cleaned_user_data(username):
  user_data=get_user_data(username)
  user_data=user_data.fillna(0)
  
  user_ep=list(user_data["my_watched_episodes"])
  for i,ep in enumerate(user_ep):
    user_ep[i]=get_series_length(ep)
  name=user_data.columns[1]
  user_data.drop(name, axis = 1, inplace = False)
  user_data[name]=user_ep
  
  rewatching=pd.to_numeric(user_data["my_rewatching"],downcast='integer')
  name=user_data.columns[3]
  user_data.drop(name, axis = 1, inplace = False)
  user_data[name]=list(rewatching)
  return user_data


In [0]:
def normalize_user(username):
  user=(cleaned_user_data(username))
  user=user.reset_index()
  #removing all plan to watch rows
  for i,status in enumerate(user["my_status"]):
    if status==6:
      user=user.drop([i])
  average=int(user["my_score"].mean())
  user_score=np.array(user)
  for i,score in enumerate(user_score[:,3]):
    loss=0
    if user_score[i,5]==4:
      loss=1
    user_score[i,3]=score-average-loss
  user["my_score"]=list(user_score[:,3])
  user=user.drop(columns=["index"])

  return user,average
norm,average=normalize_user("Xinil")



In [0]:
# start reccomender system
anime_ids_user=list(norm["anime_id"])
data_ids=list(content_filter["Anime_Id"])
#TO FIND LIST R[I][J]= 1 , if user j has rated series i
#y(i,j)= rating of user j for series i
R=[]
y=[]
for id in data_ids:
  if id in anime_ids_user:
    R.append(1)
    y.append(int(norm.loc[norm["anime_id"]==id]["my_score"]))

  else:
    R.append(0)
    y.append(12345)
m=len(R) # number of movies rated by user
content_filter["My_rating"]=y
content_filter["R_ij"]=R
content_filter=content_filter[['Anime_Id','R_ij', 'Rating','My_rating', 'Type', 'Episodes', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi',
       'Fantasy', 'Game', 'Harem', 'Historical', 'Horror',
       'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military',
       'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance',
       'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai',
       'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports',
       'Super Power', 'Supernatural', 'Thriller', 'Vampire']]#changing columns in table
user_content_filter=content_filter[content_filter.R_ij!=0]
predict_filter=content_filter[content_filter.R_ij!=1]
user_content_filter

,Anime_Id,R_ij,Rating,My_rating,Type,Episodes,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire
5,11061,1,9.11,3,8,4,0.708687,0.708687,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.708687,0.0,0.000000,0.000000,0.000000,0.708687,0.000000,0.000000,0.000000
17,2904,1,8.95,-1,8,2,0.537883,0.000000,0.000000,0.000000,0.000000,0.000000,0.537883,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.537883,0.537883,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.537883,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.537883,0.000000,0.000000,0.000000
18,199,1,8.92,4,6,1,0.000000,0.802625,0.000000,0.000000,0.000000,0.000000,0.802625,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.802625,0.000000,0.000000
26,1,1,8.81,4,8,3,0.537883,0.537883,0.000000,0.537883,0.000000,0.000000,0.537883,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.537883,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.537883,0.000000,0.000000,0.000000,0.000000,0.000000
27,164,1,8.80,3,6,1,0.802625,0.802625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.802625,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28,263,1,8.80,4,8,4,0.000000,0.000000,0.000000,0.708687,0.000000,0.000000,0.708687,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.708687,0.0,0.000000,0.000000,0.708687,0.000000,0.000000,0.000000,0.000000
29,44,1,8.79,4,4,1,0.537883,0.000000,0.000000,0.000000,0.000000,0.000000,0.537883,0.000000,0.000000,0.000000,0.0,0.537883,0.000000,0.000000,0.0,0.000000,0.537883,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.537883,0.537883,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30,1575,1,8.79,2,8,2,0.462950,0.000000,0.000000,0.000000,0.000000,0.000000,0.462950,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.462950,0.462950,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.462950,0.462950,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.462950,0.000000,0.000000,0.000000
31,12355,1,8.77,2,6,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900332,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.900332,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
36,457,1,8.74,3,8,3,0.000000,0.462950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.462950,0.000000,0.0,0.462950,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.462950,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.462950,0.000000,0.0,0.000000,0.0,0.462950,0.000000,0.000000,0.000000,0.462950,0.000000,0.000000


In [0]:
#linear regression here
X=np.array(user_content_filter.drop(columns=['Anime_Id','R_ij',""'My_rating', 'Rating']))
y=np.array(user_content_filter["My_rating"]).reshape(-1,1)

X_predict=np.array(predict_filter.drop(columns=['Anime_Id','R_ij','My_rating', 'Rating']))


In [0]:
from sklearn import datasets, linear_model

regr = linear_model.LinearRegression()
regr.fit(X, y)

# Make predictions using the testing set
y_predict = regr.predict(X_predict)


In [0]:
predict_filter["My_rating"]=y_predict
recco=pd.DataFrame(columns=["Name","Rating","Predicted_Rating"])
rating=[]
name=[]
my_rating=list(predict_filter.sort_values(by=["My_rating"],ascending=False).head(n=15)["My_rating"])

for i in list((predict_filter.sort_values(by=["My_rating"],ascending=False).head(n=15))["Anime_Id"]):
  name.append(list(df2[df2.anime_ids==i]["name"]))
  rating.append(list(df2[df2.anime_ids==i]["rating"]))
name = [item for sublist in name for item in sublist] # converting nested list into a single list (ie flattened the list)
rating = [item for sublist in rating for item in sublist] 

recco["Name"]=name
recco["Rating"]=rating
recco["Predicted_Rating"]=my_rating
recco

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Name,Rating,Predicted_Rating
0,Quanzhi Gaoshou (2018),8.36,12.480618
1,Quanzhi Gaoshou,8.08,12.480618
2,Youjo Senki,8.14,9.828434
3,Kyoukai no Kanata Movie: I&#039;ll Be Here - K...,8.00,9.199041
4,Yuri!!! on Ice: Yuri Plisetsky GPF in Barcelon...,8.12,8.630441
5,Uchuu Senkan Yamato 2202: Ai no Senshi-tachi,8.10,7.821204
6,Hikaru no Go: Journey to the North Star Cup,8.06,7.693577
7,Uchuu Senkan Yamato 2199,8.46,7.654450
8,Ginga Eiyuu Densetsu Gaiden,8.17,7.449092
9,Shelter,8.41,7.356259
